In [33]:
import pandas as pd
import numpy as np
import networkx as nx
import math
#Importing Libraries
# please do go through this python notebook: 
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd#pandas to create small dataframes 
import datetime #Convert to unix time
import time #Convert to unix time
# if numpy is not installed already : pip3 install numpy
import numpy as np#Do aritmetic operations on arrays
# matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots  
from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os
# to install xgboost: pip3 install xgboost
import xgboost as xgb

import warnings
import networkx as nx
\
import pdb
import pickle
from pandas import HDFStore,DataFrame
from pandas import read_hdf
from scipy.sparse.linalg import svds, eigs
import gc
from tqdm import tqdm

In [2]:
train_data=pd.read_csv("train_after_eda.csv")

In [3]:
test_data=pd.read_csv("test_after_eda.csv")

In [10]:
train=pd.read_csv("train.csv")
train.to_csv("traingraph.csv",header=False,index=False)

In [13]:
graph=nx.read_edgelist('traingraph.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)
print(nx.info(graph))

Name: 
Type: DiGraph
Number of nodes: 1862220
Number of edges: 9437519
Average in degree:   5.0679
Average out degree:   5.0679


# Featurization

In [15]:
def jaccard(a,b):
    try:
        if len(set(graph.successors(a)))==0 | len(set(graph.sucessors(b)))==0:
            return 0
        jac=(len(set(graph.sucessors(a)).intersection(set(graph.sucessors(b)))))/(len(set(graph.sucessors(a)).union(set(graph.sucessors(b)))))
    except:
        return 0
    return jac

In [17]:
def jaccard_followes(a,b):
    try:
        if len(set(graph.predecessors(a)))==0 | len(set(graph.predecessors(b)))==0:
            return 0
        jac=(len(set(graph.predecessors(a)).intersection(set(graph.predecessors(b)))))/(len(set(graph.predecessors(a)).union(set(graph.predecessors(b)))))
    except:
        return 0
    return jac

# Cosine Distance

In [18]:
def cosine(a,b):
    try:
        if len(set(graph.successors(a)))==0 | len(set(graph.sucessors(b)))==0:
            return 0
        cos=(len(set(graph.sucessors(a)).intersection(set(graph.sucessors(b)))))/((len(set(graph.sucessors(a))))*(len(set(graph.sucessors(b)))))
    except:
        return 0
    return cos

In [19]:
def cosine_followes(a,b):
    try:
        if len(set(graph.predecessors(a)))==0 | len(set(graph.predecessors(b)))==0:
            return 0
        cos=(len(set(graph.predecessors(a)).intersection(set(graph.predecessors(b)))))/((len(set(graph.predecessors(a))))*(len(set(graph.predecessors(b)))))
    except:
        return 0
    return cos

# Shortest Path

In [24]:
def short(a,b):
    p=-1
    try:
        if graph.has_edge(a,b):
            graph.remove_edge(a,b)
            p=nx.shortest_path_length(graph,source=a,target=b)
            graph.add_edge(a,b)
        else:
            p=nx.shortest_path_length(graph,source=a,target=b)
        return p
    except:
        return -1


# Checking for same community

In [26]:
wcc=list(nx.weakly_connected_components(graph))
def belongs_to_same_wcc(a,b):
    index = []
    if graph.has_edge(b,a):
        return 1
    if graph.has_edge(a,b):
            for i in wcc:
                if a in i:
                    index= i
                    break
            if (b in index):
                graph.remove_edge(a,b)
                if short(a,b)==-1:
                    graph.add_edge(a,b)
                    return 0
                else:
                    graph.add_edge(a,b)
                    return 1
            else:
                return 0
    else:
            for i in wcc:
                if a in i:
                    index= i
                    break
            if(b in index):
                return 1
            else:
                return 0

# Adar Index

In [28]:
def adar(a,b):
    try:
        n=list(set(graph.sucessors(a)).intersection(set(graph.sucessors(b))))
        if len(n)!=0:
            ad_sum=0
            for i in n:
                ad_sum=ad_sum+(1/np.log10(len(set(graph.predecessors(i)))))
                return ad_sum
        else:
            return 0
    except:
        return 0

# Following Back

In [30]:
def followback(a,b):
    if graph.has_edge(b,a):
        return 1
    else:
        return 0

# Katz

In [36]:
katz = pickle.load(open('katz.p','rb'))

In [37]:
print('min',katz[min(katz, key=katz.get)])
print('max',katz[max(katz, key=katz.get)])
print('mean',float(sum(katz.values())) / len(katz))

min 0.0007313532484065916
max 0.003394554981699122
mean 0.0007483800935562018


In [38]:
mean_katz = float(sum(katz.values())) / len(katz)
print(mean_katz)

0.0007483800935562018


# Hits

In [39]:
hits = pickle.load(open('hits.p','rb'))

In [40]:
print('min',hits[0][min(hits[0], key=hits[0].get)])
print('max',hits[0][max(hits[0], key=hits[0].get)])
print('mean',float(sum(hits[0].values())) / len(hits[0]))

min 0.0
max 0.004868653378780953
mean 5.615699699344123e-07


In [42]:
import random

In [43]:
n_train =  15100028
s = 100000 #desired sample size
skip_train = sorted(random.sample(range(1,n_train+1),n_train-s))

In [44]:
n_test = 3775006
s = 50000 #desired sample size
skip_test = sorted(random.sample(range(1,n_test+1),n_test-s))

In [45]:
df_final_train = pd.read_csv('train_after_eda.csv', skiprows=skip_train, names=['source_node', 'destination_node'])
df_final_train['indicator_link'] = pd.read_csv('train_y.csv', skiprows=skip_train, names=['indicator_link'])
print("Our train matrix size ",df_final_train.shape)
df_final_train.head(2)

Our train matrix size  (100002, 3)


,source_node,destination_node,indicator_link
0,273084,1505602,1
1,583350,751732,1


In [85]:
df_final_test = pd.read_csv('test_after_eda.csv', skiprows=skip_train, names=['source_node', 'destination_node'])
df_final_test['indicator_link'] = pd.read_csv('test_y.csv', skiprows=skip_train, names=['indicator_link'])
print("Our train matrix size ",df_final_test.shape)
df_final_test.head(2)

Our train matrix size  (24819, 3)


,source_node,destination_node,indicator_link
0,848424,784690,1
1,30825,270795,1


num_followers_s
num_followees_s
num_followers_d
num_followees_d
inter_followers
inter_followees

In [47]:
from tqdm import tqdm

In [58]:
followers_s=[]
followees_s=[]
followers_d=[]
followees_d=[]
inter_followers=[]
inter_followees=[]
source=df_final_train["source_node"]
destination=df_final_train["destination_node"]

for i in tqdm(range(len(source))):
    try:
        s1=set(graph.predecessors(source[i]))
        s2=set(graph.sucessors(source[i]))
    except:
        s1=set()
        s2=set()
    try:
        d1=set(graph.predecessors(destination[i]))
        d2=set(graph.sucessors(destination[i]))
    except:
        d1=set()
        d2=set()
    followers_s.append(len(s1))
    followees_s.append(len(s2))
    followers_d.append(len(d1))
    followees_d.append(len(d2))
    inter_followers.append(len(s1.intersection(d1)))
    inter_followees.append(len(s2.intersection(d2)))
    

100%|███████████████████████████████████████████████████████████████████████| 100002/100002 [00:03<00:00, 27439.20it/s]


# weight

In [59]:
Weight_in = {}
Weight_out = {}
for i in  tqdm(graph.nodes()):
    s1=set(graph.predecessors(i))
    w_in = 1.0/(np.sqrt(1+len(s1)))
    Weight_in[i]=w_in
    
    s2=set(graph.successors(i))
    w_out = 1.0/(np.sqrt(1+len(s2)))
    Weight_out[i]=w_out
    
#for imputing with mean
mean_weight_in = np.mean(list(Weight_in.values()))
mean_weight_out = np.mean(list(Weight_out.values()))

100%|█████████████████████████████████████████████████████████████████████| 1862220/1862220 [00:23<00:00, 79468.99it/s]


# SVD

In [60]:
def svd(x, S):
    try:
        z = sadj_dict[x]
        return S[z]
    except:
        return [0,0,0,0,0,0]

In [62]:
sadj_col = sorted(graph.nodes())
sadj_dict = { val:idx for idx,val in enumerate(sadj_col)}

In [64]:
Adj = nx.adjacency_matrix(graph,nodelist=sorted(graph.nodes())).asfptype()

In [65]:
U, s, V = svds(Adj, k = 6)
print('Adjacency matrix Shape',Adj.shape)
print('U Shape',U.shape)
print('V Shape',V.shape)
print('s Shape',s.shape)

Adjacency matrix Shape (1862220, 1862220)
U Shape (1862220, 6)
V Shape (6, 1862220)
s Shape (6,)


# Adding Features to Training data

In [86]:
df_final_train['jaccard_followers'] = df_final_train.apply(lambda row: jaccard(row['source_node'],row['destination_node']),axis=1)

,source_node,destination_node,indicator_link
0,848424,784690,1
1,30825,270795,1


In [ ]:
df_final_train["jaccard_followees"]=df_final_train.apply(lambda row: jaccard_followes(row['source_node'],row['destination_node']),axis=1)
df_final_train["cosine_followers"]=df_final_train.apply(lambda row: cosine(row["source_node"],row["destination_node"]),axis=1)
df_final_train["cosine_followes"]=df_final_train.apply(lambda row: cosine_followes(row["source_node"],row["destination_node"]),axis=1)
df_final_train["shortest_path"]=df_final_train.apply( lambda row: short(row["source_node"],row["destination_node"]),axis=1)
df_final_train["same_commu"]=df_final_train.apply( lambda row: belongs_to_same_wcc(row["source_node"],row["destination_node"]),axis=1)


In [ ]:
df_final_train["adar"]=df_final_train.apply( lambda row: adar(row["source_node"],row["destination_node"]),axis=1)
df_final_train["followback"]=df_final_train.apply( lambda row: followback(row["source_node"],row["destination_node"]),axis=1)
df_final_train['katz_s'] = df_final_train.source_node.apply(lambda x: katz.get(x,mean_katz))
df_final_train['katz_d'] = df_final_train.destination_node.apply(lambda x: katz.get(x,mean_katz))

df_final_train['hubs_s'] = df_final_train.source_node.apply(lambda x: hits[0].get(x,0))
df_final_train['hubs_d'] = df_final_train.destination_node.apply(lambda x: hits[0].get(x,0))



In [ ]:
df_final_train['authorities_s'] = df_final_train.source_node.apply(lambda x: hits[1].get(x,0))
df_final_train['authorities_d'] = df_final_train.destination_node.apply(lambda x: hits[1].get(x,0))



In [ ]:
df_final_train['weight_in'] = df_final_train.destination_node.apply(lambda x: Weight_in.get(x,mean_weight_in))
df_final_train['weight_out'] = df_final_train.source_node.apply(lambda x: Weight_out.get(x,mean_weight_out))

df_final_train['weight_f1'] = df_final_train.weight_in + df_final_train.weight_out
df_final_train['weight_f2'] = df_final_train.weight_in * df_final_train.weight_out
df_final_train['weight_f3'] = (2*df_final_train.weight_in + 1*df_final_train.weight_out)
df_final_train['weight_f4'] = (1*df_final_train.weight_in + 2*df_final_train.weight_out)

 

In [ ]:
df_final_train[['svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6']] = \
df_final_train.source_node.apply(lambda x: svd(x, U)).apply(pd.Series)
    
df_final_train[['svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6']] = \
df_final_train.destination_node.apply(lambda x: svd(x, U)).apply(pd.Series)
    #===================================================================================================
    
df_final_train[['svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6',]] = \
df_final_train.source_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)

df_final_train[['svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6']] = \
df_final_train.destination_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)

# Test Data

In [ ]:
df_final_test['jaccard_followers'] = df_final_test.apply(lambda row: jaccard(row['source_node'],row['destination_node']),axis=1)

In [ ]:
df_final_test["jaccard_followees"]=df_final_test.apply(lambda row: jaccard_followes(row['source_node'],row['destination_node']),axis=1)
df_final_test["cosine_followers"]=df_final_test.apply(lambda row: cosine(row["source_node"],row["destination_node"]),axis=1)
df_final_test["cosine_followes"]=df_final_test.apply(lambda row: cosine_followes(row["source_node"],row["destination_node"]),axis=1)
df_final_test["shortest_path"]=df_final_test.apply( lambda row: short(row["source_node"],row["destination_node"]),axis=1)
df_final_test["same_commu"]=df_final_test.apply( lambda row: belongs_to_same_wcc(row["source_node"],row["destination_node"]),axis=1)


In [ ]:
df_final_test["adar"]=df_final_test.apply( lambda row: adar(row["source_node"],row["destination_node"]),axis=1)
df_final_test["followback"]=df_final_test.apply( lambda row: followback(row["source_node"],row["destination_node"]),axis=1)
df_final_test['katz_s'] = df_final_test.source_node.apply(lambda x: katz.get(x,mean_katz))
df_final_test['katz_d'] = df_final_test.destination_node.apply(lambda x: katz.get(x,mean_katz))

df_final_test['hubs_s'] = df_final_test.source_node.apply(lambda x: hits[0].get(x,0))
df_final_test['hubs_d'] = df_final_test.destination_node.apply(lambda x: hits[0].get(x,0))



In [ ]:
df_final_test['authorities_s'] = df_final_test.source_node.apply(lambda x: hits[1].get(x,0))
df_final_test['authorities_d'] = df_final_test.destination_node.apply(lambda x: hits[1].get(x,0))



In [ ]:
df_final_test['weight_in'] = df_final_test.destination_node.apply(lambda x: Weight_in.get(x,mean_weight_in))
df_final_test['weight_out'] = df_final_test.source_node.apply(lambda x: Weight_out.get(x,mean_weight_out))

df_final_test['weight_f1'] = df_final_test.weight_in + df_final_test.weight_out
df_final_test['weight_f2'] = df_final_test.weight_in * df_final_test.weight_out
df_final_test['weight_f3'] = (2*df_final_test.weight_in + 1*df_final_test.weight_out)
df_final_test['weight_f4'] = (1*df_final_test.weight_in + 2*df_final_test.weight_out)

 

In [ ]:
df_final_test[['svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6']] = \
df_final_test.source_node.apply(lambda x: svd(x, U)).apply(pd.Series)
    
df_final_test[['svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6']] = \
df_final_test.destination_node.apply(lambda x: svd(x, U)).apply(pd.Series)
    #===================================================================================================
    
df_final_test[['svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6',]] = \
df_final_test.source_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)

df_final_test[['svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6']] = \
df_final_test.destination_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)

In [95]:
df_final_train.to_csv("final_train.csv",index=False)
df_final_test.to_csv("final_test.csv",index=False)